In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net=nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X=torch.rand(2,20)
print(X)
print(net(X))

tensor([[0.2243, 0.3840, 0.8949, 0.6740, 0.4546, 0.2304, 0.1483, 0.8898, 0.9775,
         0.7477, 0.4662, 0.6672, 0.6994, 0.1178, 0.3331, 0.9812, 0.5006, 0.3022,
         0.9212, 0.2956],
        [0.1418, 0.9334, 0.0509, 0.5866, 0.4924, 0.7301, 0.0022, 0.2459, 0.6182,
         0.8326, 0.6727, 0.5267, 0.2939, 0.3149, 0.4983, 0.6196, 0.9784, 0.3597,
         0.3546, 0.9738]])
tensor([[ 0.1778,  0.0387, -0.4088,  0.1233,  0.1461, -0.1431,  0.2253,  0.0914,
         -0.1323, -0.1203],
        [ 0.2680,  0.1550, -0.2162,  0.1399,  0.1856, -0.1300,  0.0714,  0.1149,
         -0.1651, -0.2014]], grad_fn=<AddmmBackward0>)


In [2]:
#自定义块
class MLP(nn.Module):   #多层感知机
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))        

In [2]:
#自定义块
class MLP(nn.Module):   #多层感知机
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))        

In [4]:
net=MLP()
net(X)

tensor([[-0.0622,  0.2228,  0.0411,  0.1117,  0.0425, -0.0948, -0.2438, -0.3385,
          0.0884,  0.2098],
        [ 0.1398,  0.0869,  0.3103,  0.1366, -0.0223, -0.0288, -0.2343, -0.2064,
          0.1030,  0.2396]], grad_fn=<AddmmBackward0>)

In [6]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            self._modules[str(idx)]=module
    
    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X

In [7]:
net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.4336,  0.1523, -0.2063,  0.2644,  0.1875,  0.4090,  0.0187,  0.0654,
         -0.1037,  0.2460],
        [-0.1482,  0.1741, -0.2710,  0.1752,  0.0369,  0.2524,  0.0423,  0.1025,
         -0.0267,  0.0479]], grad_fn=<AddmmBackward0>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
    
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        X=self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()

In [9]:
net=FixedHiddenMLP()
net(X)

tensor(0.3497, grad_fn=<SumBackward0>)

In [13]:
class NetMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),
                              nn.ReLU(),
                              nn.Linear(64,32),
                              nn.ReLU())
        self.linear=nn.Linear(32,16)
        
    def forward(self,X):
        return self.linear(self.net(X))
    
chimera=nn.Sequential(NetMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(0.0531, grad_fn=<SumBackward0>)